In [8]:
#connecting to the drive
from google.colab import drive
drive.mount('/content/drive/')


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
#Headers and libraries

import numpy as np
from keras.models import Model
from keras.layers import *
import keras.backend as K
from keras.layers import Input
from keras.models import Model
from keras.optimizers import Adam
import datetime
from os import path, makedirs
from keras.callbacks import TensorBoard
import tensorflow as tf
import random
import cv2

In [0]:


class DataLoader():
    """
    :dataset: string
    :name_train: string
    :name_mask: string
    :val_img: string
    :val_msk: string
    :test: string
    :rtype: numpy arrays (float32)
    
    This class deals with all the data related calls. The class is called by DataLoader(Path to the folder where you have the data,
    name of the train file, name of the train mask, name of validation image file, name of the validation mask file, name of the test file)
    All the files are numpy arrarys stored in .npy format
    """

    def __init__(self, dataset,name_train='',name_mask='',val_img='',val_msk='',test=''):
        self.dataset = dataset
        self.image=name_train
        self.mask=name_mask
        self.val_msk=val_msk
        self.val_img=val_img
        self.tst=test
        
        
    def load_test(self):
        return np.load(dataset+test)


    def load_batch(self, batch_size=1):
        imgs_trn = np.load(self.dataset+'/' + self.image)
        msks_trn = np.load(self.dataset+'/' + self.mask)
        self.n_batches = int(imgs_trn.shape[0] / batch_size)
        
        for i in range(self.n_batches - 1):
            imgs =imgs_trn[i * batch_size:(i + 1) * batch_size]
            labels = msks_trn[i * batch_size:(i + 1) * batch_size]
            yield imgs, labels

            
    def load_img(self, batch_size=1):
        imgs_val = np.load(self.dataset+'/'+ self.image)
        msks_val = np.load(self.dataset +'/'+ self.mask)
        batch_images = random.sample(range(imgs_val.shape[0]), batch_size)
        return imgs_val[batch_images],msks_val[batch_images]



In [0]:
"""
this is the dice coefficient loss which is used for training along side mse. 
EQN used: 2*sum(a.b)/sum(a)+sum(b) this is dice coef for our purpose.
1-dice coeff is the loss.

"""
def dice_coef(y_true, y_pred):
    smooth = 1.0
    y_true_flat = K.flatten(y_true)
    y_pred_flat = K.flatten(y_pred)
    intersection = K.sum(y_true_flat * y_pred_flat)
    return (2.0 * intersection + smooth) / (K.sum(y_true_flat) + K.sum(y_pred_flat) + smooth)

  
def dice_coef_loss(y_true, y_pred):
    return 1 - 1.0 * dice_coef(y_true, y_pred)


In [0]:
#network
"""
The get_densenet takes input shape and builds a dense dilated network. 
The feature size and dpeth are set to 4 which can be reduced in case of lower computational power.
the build_discriminator function takes input size and discriminator factor as input the value DF is then used to determine the size of the layers.
"""


def get_densenet(input_shape,depth,feature=48):
    '''
    :input_shape: (int,int,int)
    :depth: int 
    :features: int (preferably 48)
    :rtype: untrained model
    
    The feature size is the same as the one in the code base. Do not reduce the factor below 12 it results in poor feature extraction.
    The depth can be adjusted according to need. Do not reduce below 2. you can go higher but would be computationally heavy.
    
    '''
    K.set_image_dim_ordering('tf')
    x = inputs = Input(shape=input_shape, dtype='float32')
    x = Conv2D(feature, 3, padding='same', activation='relu', kernel_initializer='he_normal')(x)
    x = Dropout(0.2)(x)
    x = dc_0_out = dense_block(x,depth=depth)
    x = transition_Down(x, feature*2)
    x = dc_1_out = dense_block(x,depth=depth)
    x = transition_Down(x, feature*3)
    x = dc_2_out = dense_block(x,depth=depth)
    x = transition_Down(x, feature*4)
    x = dc_3_out = dense_block(x,depth=depth)
    x = transition_Down(x, feature*5)
    x = dense_block(x,depth=depth)
    x = transition_Up(x, feature)
    x = concatenate([x, dc_3_out])
    x = dense_block(x,depth=depth)
    x = transition_Up(x, feature)
    x = concatenate([x, dc_2_out])
    x = dense_block(x,depth=depth)
    x = transition_Up(x, feature)
    x = concatenate([x, dc_1_out])
    x = dense_block(x,depth=depth)
    x = transition_Up(x, feature)
    x = concatenate([x, dc_0_out])
    x = dense_block(x,depth=depth)
    x = Conv2D(1, 1, activation='sigmoid')(x)
    net = Model(inputs=inputs, outputs=x)
    return net

  
def dense_block(input_layer, features=12, depth=4,temperature=1.0, padding='same', batchnorm=False,dropout=0.2):
    '''
    :input_layer: tensor object
    :features: int
    :depth: int 
    :temperature: float
    :padding: string 
    :batchnorm: bool
    :dropout: float
    :rtype: a block of layers 
    
    This is where we create the dense dilation blocks. 
    '''
    inputs = x = input_layer
    maps = [inputs]
    dilation_rate = 1
    kernel_size = (3, 3)
    for n in range(depth):
        x0 = x
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Conv2D(features, kernel_size, dilation_rate=dilation_rate,
                   padding=padding, kernel_initializer='he_normal')(x)
        x = Dropout(dropout)(x)
        maps.append(x)
        if n!= depth-1:
            x = Concatenate()([x0, x])
        else:
            x = Concatenate()(maps)
        dilation_rate *= 2
    return x


def transition_Down(input_layer,features,kernel_size=(3,3), padding='same',dropout=0.2):
    '''
    :input_layer: tensor object
    :features: int
    :kernel_size: (int,int)
    :padding: string
    :dropout: float
    :rtype: tensor object
    
    This is the downsampling. 
    '''
    x = BatchNormalization()(input_layer)
    x = Activation('relu')(x)
    x = Conv2D(features, kernel_size,
                   padding=padding, kernel_initializer='he_normal')(x)
    x = Dropout(dropout)(x)
    x = MaxPooling2D(2,2)(x)
    return x

  
def transition_Up(input_layer, feature, kernel_size=(3,3),stride=2):
    '''
    :input_layer: tensor object
    : features: int
    :kernel_size: (int,int)
    :stride: int
    :rtype: tensor object
    
    This upsamples the feature block.
    '''
    x = Conv2DTranspose(feature, 2, strides=stride, activation='relu', kernel_initializer='he_normal')(input_layer)
    return x

  
def build_discriminator(img_shape,df):
    '''
    :img_shape: (int,int,int)
    :df: int
    :rtype: untrained model
    
    This builds the discriminator network  with a custom feature size decided by df. 
    '''
    def d_layer(layer_input, filters, f_size=4, bn=True):
        """Discriminator layer"""
        d = Conv2D(filters, kernel_size=f_size, strides=2, padding='same')(layer_input)
        d = LeakyReLU(alpha=0.2)(d)
        if bn:
            d = BatchNormalization(momentum=0.8)(d)
        return d
    def layer (layer_input, filters, f_size=4, bn=True):
        d = Conv2D(filters, kernel_size=f_size, strides=1, padding='same')(layer_input)
        d = LeakyReLU(alpha=0.2)(d)
        if bn:
            d = BatchNormalization(momentum=0.8)(d)
        return d
    img_A = Input(shape=img_shape)
    img_B = Input(shape=img_shape)

    # Concatenate image and conditioning image by channels to produce input
    combined_imgs = Concatenate(axis=-1)([img_A, img_B])
    d1 = d_layer(combined_imgs, df, bn=False)
    d2 = d_layer(d1, df*2)
    d3 = layer(d2, df*4)
    # d4 = layer(d3, df*8)
    validity = Conv2D(1, kernel_size=4, strides=1, padding='same')(d3)
    model = Model([img_A, img_B], validity)
    return model

In [0]:
#main

class ADDN():
    """
    :checkpoint_name: string
    :data: dict 
    
    THe ADDN class is where training and testing is done. 
    Compile is where the model is initialised and compiled and train module trains the GAN.
    Sample images is used to store imaged while training and saveResult is used to save predicted output during testing.
    """
    def __init__(self,checkpoint_name,data):
        ### Configurations
        self.config = data
        # Calculate output shape of D (PatchGAN)
        self.img_rows = int(self.config['input_shape'][0])
        self.disc_patch = (self.config['patch'], self.config['patch'], 1)
        self.data_loader = DataLoader(dataset=self.config['data_path'],name_train=self.config['train_image_name'],name_mask=self.config['train_mask_name'],test=self.config['test_image_name'])
        self.checkpoint_name = checkpoint_name

        self.generator = None
        self.discriminator = None
        self.combined = None
        self.imgs_trn = None
        self.msks_trn = None
        self.imgs_val = None
        self.msks_val = None
        log_path = 'Graph/addn'
        self.callback = TensorBoard(log_path)
        return

    
    def checkpoint_path(self):
        return self.config['data_path']+'%s' % (self.checkpoint_name)

      
    def compile(self):
        optimizer = Adam(0.0002, 0.5)
        # Build and compile the discriminator
        self.discriminator = build_discriminator(self.config['input_shape'], self.config['df'])
        self.discriminator.compile(loss='mse', optimizer=optimizer, metrics=['accuracy'])
        self.discriminator.summary()
        # Build the generator
        self.generator = get_densenet(self.config['input_shape'],self.config['depth'],self.config['generator_factor'])
        self.generator.summary()
        #self.generator =get_generator(self.config['input_shape'])
        img = Input(shape=self.config['input_shape'])
        label = Input(shape=self.config['input_shape'])
        seg = self.generator(img)
        self.discriminator.trainable = False
        valid = self.discriminator([seg, img])
        self.combined = Model(inputs=[label, img], outputs=[valid, seg])
        self.combined.compile(loss=['mse',dice_coef_loss], loss_weights=[1, 100], optimizer=optimizer)
        #self.callback.set_model(self.generator)
        return

      
    def train(self, sample=False):
        start_time = datetime.datetime.now()
        # Adversarial loss ground truths
        valid = np.ones((self.config['batch_size'],) + self.disc_patch)
        fake = np.zeros((self.config['batch_size'],) + self.disc_patch)
        gen_loss=[]
        dis_loss=[]
        acc=[]
        for epoch in range(self.config['epochs']):
            for batch_i, (imgs, labels) in enumerate(self.data_loader.load_batch(self.config['batch_size'])):
                # Condition on B and generate a translated version

                # Train the discriminators (original images = real / generated = Fake)
                dl=100000
                segs = self.generator.predict(imgs)
                d_loss_real = self.discriminator.train_on_batch([labels, imgs], valid)
                d_loss_fake = self.discriminator.train_on_batch([segs, imgs], fake)
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
                # Train the generators
                g_loss = self.combined.train_on_batch([labels, imgs], [valid, labels])
                elapsed_time = datetime.datetime.now() - start_time
                # Plot the progress
                print ("[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %3d%%] [G loss: %f] time: %s" % (epoch, self.config['epochs'],
                                                                        batch_i, self.data_loader.n_batches,
                                                                        d_loss[0], 100*d_loss[1],
                                                                        g_loss[0],
                                                                        elapsed_time))
                if dl>d_loss[0]:
                  self.generator.save_weights(self.config['weights_path_with_name'])
                  dl=d_loss[0]
                  ac=acc

                # If at save interval => save generated image samples
                if sample == True:
                    if batch_i % self.config['sample_interval'] == 0:
                        self.sample_images(epoch, batch_i)
            train_names = 'train_loss'
            
            gen_loss.append(g_loss)
            dis_loss.append(d_loss)
            acc.append(100*d_loss[1])
        np.save(self.config['savepath']+'/discriminator.npy',dis_loss)
        np.save(self.config['savepath']+'/generator.npy',gen_loss)
        np.save(self.config['savepath']+'/accuracy.npy',acc)
            
        print(dl,ac)
        return [gen_loss,dis_loss,acc]
    
    
    def predict(self, imgs):
        return self.generator.predict(imgs)

      
    def sample_images(self, epoch, batch_i):
        # r, c = 3, 3
        imgs, labels = self.data_loader.load_img(batch_size=1)
        segs = self.predict(imgs)
        print (segs.shape)
        cv2.imwrite(self.config['save_train_images']+'/%d_%d.bmp' % (epoch, batch_i), segs[0][:, :, 0]*255)
    
    
    def test(self):
          imgtst=self.data_loader.load_test()
          #print(imgtst.shape) 

          results=self.generator.predict(imgtst,batch_size=2, verbose=1)

          def saveResult(self,npyfile):
              for i in range(numpyfile.shape[0]):
                  cv2.imwrite(self.config['save_path']+"%d_predict_vm.png"%i,npyfile[i]*255)
          self.saveResult(self.config['savepath'],results)

In [14]:
"""
:data:{
        'data_path': path to the folder which has the data        
        'train_image_name':name of the train image file
        'train_mask_name':name of the train mask file
        'val_image_name':name of the validation image file
        'val_mask_name':name of the validation mask file
        'test_image_name':name of the test file
        "weights_path_with_name":'path to the loaction where you want to save the weights/nameof the file.hd5
        'save_train_images':path to loaction where you want to save the images generated during training to check the progress
        'savepath':path to loaction where you want to save the predicted output after training
        'depth': depth of the dilated network
        'input_shape': Input shape
        'output_shape': output shape
        'batch_size': batch size. If the model keep having memory error reduce the batch size. For 512x512 set to 1 if running on Colab.
        'epochs': number of epochs
        'sample_interval': frequency of posting images generated during training 
        'df':64, Discriminator factor. this decides the size of the layers of discriminator
        'patch':this is  for stabalising the model. decided by imagesize/4. for 512-->128, 128-->32
        'feature_factor': feature factor for generator. 
        }



"""


data={
        'data_path': "drive/My Drive/GAN_seg/ADDN-master/data",
        'save_train_images':'drive/My Drive/GAN_seg/ADDN-master/vm/train',
        'train_image_name':'/new_img_trn.npy',
        'train_mask_name':'/new_msk_trn.npy',
        'val_image_name':"",
        'val_mask_name':'',
        'test_image_name':'/img_tst.npy',
        "weights_path_with_name":'drive/My Drive/GAN_seg/ADDN-master/vm/checkpoints/pack.hd5',
        'savepath':'drive/My Drive/GAN_seg/ADDN-master/vm/predict', 
        'depth':4,
        'input_shape': (512, 512, 1),
        'output_shape': (512, 512, 1),
        'batch_size': 1,
        'epochs': 5,
        'sample_interval': 200,
        'df':64,
        'patch':128,
        'generator_factor':48
        }

model=ADDN('trail',data)
model.compile()
gen,dis,acc= model.train(sample=True)
model.test()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 512, 512, 1)  0                                            
__________________________________________________________________________________________________
input_7 (InputLayer)            (None, 512, 512, 1)  0                                            
__________________________________________________________________________________________________
concatenate_42 (Concatenate)    (None, 512, 512, 2)  0           input_6[0][0]                    
                                                                 input_7[0][0]                    
__________________________________________________________________________________________________
conv2d_47 (Conv2D)              (None, 256, 256, 64) 2112        concatenate_42[0][0]             
__________

KeyboardInterrupt: ignored